# 預測用戶在2016年6月領取優惠券後15天以內的使用情況
***
- 分數以網站評分結果為準, 請同學實際將提交檔(*.csv)上傳試試看  
https://www.kaggle.com/c/ml100marathon-02-01/overview

# [欄位說明]
* User_id：用戶 ID 
* Merchant_id：商家 ID 
* Coupon_id：優惠券 ID (null 代表無優惠券消費) 
* Discount_rate：優惠券折價：[0,1] 代表折扣率；x:y 代表滿 x 減 y 元 
* Distance：用戶經常活動地點離商家最近距離 (x * 500 公尺), 0 表示低於 500 公尺, 10 表示大於 5 公里。 
* Date_received：優惠券取得時間。 
* Date：購買商品時間 (如果 Date is null & Coupon_id is not null, 則該紀錄為有優惠券但未使用; 若為 Date is not null & Coupon_id is null, 則為普通消費日期; 若 Date is not null & Coupon_id is not null, 則表示優惠券消費日期)

In [1]:
# 做完特徵工程前的所有準備
import os
import numpy as np
import pandas as pd
from datetime import date, datetime

from sklearn.model_selection import KFold, train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve
from sklearn.preprocessing import MinMaxScaler

data_path = './data/D51-D53：Kaggle期中考/'

# load data
df_train = pd.read_csv(data_path + 'train_offline.csv')
df_test = pd.read_csv(data_path + 'test_offline.csv')
df_test = df_test[~df_test.Coupon_id.isna()]
df_test.reset_index(drop=True, inplace=True)

print("df_train shape: ", df_train.shape)
print("df_test shape: ", df_test.shape)

df_train.head()

df_train shape:  (1160742, 7)
df_test shape:  (306313, 6)


,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN


In [2]:
## Creat target label 
"""
According to the definition, 
1) buy with coupon within (include) 15 days ==> 1
2) buy with coupon but out of 15 days ==> 0
3) buy without coupon ==> -1 (we don't care)
"""
def label(row):
    data_received = row['Date_received']
    if(np.isnan(row['Date_received'])):
        return -1
    if(np.isnan(row['Date'])):
        return 0
    else:
        sub_time = pd.to_datetime(row['Date'], format='%Y%m%d') -  pd.to_datetime(row['Date_received'], format='%Y%m%d')
        return 1 if sub_time <= pd.Timedelta(15, 'D') else 0
    
df_train['Label'] = df_train.apply(label, axis=1)
df_train['Label'].value_counts()

 0    710665
-1    413773
 1     36304
Name: Label, dtype: int64

## 第一步驟：處理特徵值

#### 1. 星期特徵值

In [3]:
# Generate features - weekday acquired coupon
def getWeekday(row):
    if (np.isnan(row)) or (row==-1):
        return row
    else:
        return pd.to_datetime(row, format = "%Y%m%d").dayofweek+1 # add one to make it from 0~6 -> 1~7

df_train['weekday'] = df_train['Date_received'].apply(getWeekday)
df_test['weekday'] = df_test['Date_received'].apply(getWeekday)

# weekday_type (weekend = 1)
df_train['weekday_type'] = df_train['Date_received'].astype('str').apply(lambda x: 1 if x in [6,7] else 0)
df_test['weekday_type'] = df_test['Date_received'].astype('str').apply(lambda x: 1 if x in [6,7] else 0)

In [4]:
# 將weekday 做 label enconding
weekdaycols = ['weekday_' + str(i) for i in range(1,8)]
print(weekdaycols)

tmp_train_df = pd.get_dummies(df_train['weekday'].replace(-1, np.nan))
tmp_train_df.columns = weekdaycols
df_train[weekdaycols] = tmp_train_df

tmp_test_df = pd.get_dummies(df_test['weekday'].replace(-1, np.nan))
tmp_test_df.columns = weekdaycols
df_test[weekdaycols] = tmp_test_df

['weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']


#### 2. discount and distance 特徵值

In [5]:
# Generate features - coupon discount and distance
def getDiscountType(row):
    if row == 'null':
        return 'null'
    elif ':' in row:
        return 1
    else:
        return 0

def convertRate(row):
    """Convert discount to rate"""
    if row == 'null':
        return 1.0
    elif ':' in row:
        rows = row.split(':')
        return 1.0 - float(rows[1])/float(rows[0])
    else:
        return float(row)

def getDiscountMan(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[0])
    else:
        return 0

def getDiscountJian(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[1])
    else:
        return 0

def process_discount_data(df):
    #convert discunt_rate
    df['discount_rate'] = df['Discount_rate'].astype('str').apply(convertRate)
    df['discount_man'] = df['Discount_rate'].astype('str').apply(getDiscountMan)
    df['discount_jian'] = df['Discount_rate'].astype('str').apply(getDiscountJian)
    df['discount_type'] = df['Discount_rate'].astype('str').apply(getDiscountType)
    
    # convert distance
    df.loc[df.Distance.isna(), "Distance"] = 99
    return df

df_train = process_discount_data(df_train)
df_test = process_discount_data(df_test)

#### 3. 日期型別

In [6]:
def analysis_date(row):
    if(np.isnan(row) or row == ''):
        return 'not value'
    else:
        data_received = datetime.strptime(str(round(row)), '%Y%m%d')
        return data_received

def get_date_result(row, state = 'year'):
    if(row == 'not value'):
        return 0
    elif(state == 'year'):
        return datetime.strftime(row, '%Y')
    elif(state == 'month'):
        return datetime.strftime(row, '%m')
    else:
        return datetime.strftime(row, '%d')
    
def get_date_type(row):
    if(row == 'not value'):
        return 0
    else:
        return 1
    
def process_Date_data(df):
    analysis_result = df['Date_received'].apply(analysis_date).astype('object')
    df['year'] = analysis_result.apply(lambda x :get_date_result(x, 'year')).astype('int64')
    df['month'] = analysis_result.apply(lambda x :get_date_result(x, 'month')).astype('int64')
    df['day'] = analysis_result.apply(lambda x :get_date_result(x, 'day')).astype('int64')
    df['date_type'] = analysis_result.apply(lambda x: 1 if x != 'not value' else 0)
    return df

df_train = process_Date_data(df_train)
df_test = process_Date_data(df_test)

In [7]:
df_train.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,Label,weekday,weekday_type,...,weekday_6,weekday_7,discount_rate,discount_man,discount_jian,discount_type,year,month,day,date_type
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0,-1,NaN,0,...,0,0,NaN,0,0,0,0,0,0,0
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,0,3.0,0,...,0,0,0.95,20,1,1,2016,2,17,1
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,0,6.0,0,...,1,0,0.95,20,1,1,2016,3,19,1
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,0,5.0,0,...,0,0,0.90,200,20,1,2016,4,29,1
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN,0,5.0,0,...,0,0,0.90,200,20,1,2016,1,29,1


# 第二步驟：切割資料及為 train & valid data

In [8]:
## Naive model
def split_train_valid(row, date_cut="20160416"):
    is_train = True if pd.to_datetime(row, format="%Y%m%d") < pd.to_datetime(date_cut, format="%Y%m%d") else False
    return is_train

df = df_train[df_train['Label'] != -1].copy()
df["is_train"] = df["Date_received"].apply(split_train_valid)

train = df[df["is_train"]] ; train.reset_index(drop=True, inplace=True)
val = df[~df["is_train"]] ; val.reset_index(drop=True, inplace=True)

print("Train size: {}, #positive: {}".format(len(train), train["Label"].sum()))
print("Valid size: {}, #positive: {}".format(len(val), val["Label"].sum()))

Train size: 667753, #positive: 32472
Valid size: 79216, #positive: 3832


# 第三步驟：model trainging

In [9]:
original_feature = ['discount_rate',
                    'discount_type',
                    'discount_man', 
                    'discount_jian',
                    'Distance', 
                    'weekday', 
                    'weekday_type'] + weekdaycols + ['year', 'month', 'day', 'date_type']
print(len(original_feature), original_feature)

18 ['discount_rate', 'discount_type', 'discount_man', 'discount_jian', 'Distance', 'weekday', 'weekday_type', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7', 'year', 'month', 'day', 'date_type']


In [10]:
predictors = original_feature
print(predictors)

def check_model(data, predictors):
    
    classifier = lambda: SGDClassifier(
        loss='log', 
        penalty='elasticnet', 
        fit_intercept=True, 
        max_iter=100, 
        shuffle=True, 
        n_jobs=1,
        class_weight=None)

    model = Pipeline(steps=[
        ('ss', StandardScaler()),
        ('en', classifier())
    ])

    parameters = {
        'en__alpha': [ 0.001, 0.01, 0.1],
        'en__l1_ratio': [ 0.001, 0.01, 0.1]
    }

    folder = StratifiedKFold(n_splits=3, shuffle=True)

    grid_search = GridSearchCV(
        model, 
        parameters, 
        cv=folder, 
        n_jobs=-1, 
        verbose=1)
    grid_search = grid_search.fit(data[predictors], 
                                  data['Label'])
    
    return grid_search

model = check_model(train, predictors)

['discount_rate', 'discount_type', 'discount_man', 'discount_jian', 'Distance', 'weekday', 'weekday_type', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7', 'year', 'month', 'day', 'date_type']
Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:   17.5s finished


In [11]:
y_val_pred = model.predict_proba(val[predictors])
val1 = val.copy()
val1['pred_prob'] = y_val_pred[:, 1]

In [12]:
from sklearn.metrics import roc_auc_score, accuracy_score
auc_score = roc_auc_score(y_true=val.Label, y_score=y_val_pred[:,1])
acc = accuracy_score(y_true=val.Label, y_pred=y_val_pred.argmax(axis=1))
print("Validation AUC: {:.3f}, Accuracy: {:.3f}".format(auc_score, acc))

Validation AUC: 0.745, Accuracy: 0.952


# 第四步驟：test predict

#### 1. 預測測試集資料

In [13]:
target_set = df_test.copy()
print(target_set.shape)
target_set = target_set[~target_set.Coupon_id.isna()]
target_set.reset_index(drop=True, inplace=True)
test_set = target_set[predictors].copy()

y_test_pred = model.predict_proba(test_set[predictors])
test1 = test_set.copy()
test1['pred_prob'] = y_test_pred[:, 1]
print(test1.shape)

(306313, 23)
(306313, 19)


#### 2. 處理uid

In [14]:
output = pd.concat((target_set[["User_id", "Coupon_id", "Date_received"]], test1["pred_prob"]), axis=1)
print(output.shape)

output.loc[:, "User_id"] = output["User_id"].apply(lambda x:str(int(x)))
output.loc[:, "Coupon_id"] = output["Coupon_id"].apply(lambda x:str(int(x)))
output.loc[:, "Date_received"] = output["Date_received"].apply(lambda x:str(int(x)))
output["uid"] = output[["User_id", "Coupon_id", "Date_received"]].apply(lambda x: '_'.join(x.values), axis=1)
output.reset_index(drop=True, inplace=True)

(306313, 4)


#### 3. output result csv

In [15]:
### NOTE: YOUR SUBMITION FILE SHOULD HAVE COLUMN NAME: uid, label
out = output.groupby("uid", as_index=False).mean()
out = out[["uid", "pred_prob"]]
out.columns = ["uid", "label"]
out.to_csv("./example/D51-D53：Kaggle期中考/baseline_example.csv", header=["uid", "label"], index=False) # submission format
out.head()

,uid,label
0,1000020_2705_20160519,0.293953
1,1000020_8192_20160513,0.237244
2,1000065_1455_20160527,0.228069
3,1000085_8067_20160513,0.196744
4,1000086_2418_20160613,0.276685
